In [1]:
# -*- coding: utf-8 -*-
from unittest import TestCase
from correios_lib.webservices import LogisticaReversa
from correios_lib.entities import Remetente, Destinatario, Coleta, \
     Objeto, Produto, ColetaSimultanea
from correios_lib.requests import RequestSolicitarPostagemReversa, \
     RequestAcompanharPedido, RequestAcompanharPedidoPorData, \
     RequestCancelarPedido, RequestSolicitarRange, \
     RequestCalcularDigitoVerificador, RequestSolicitarPostagemSimultanea
import datetime

In [2]:
cliente=LogisticaReversa(
    env='PROD',
    id_correios='grupo.sbf',
    password='centauro2021',
    cert=True)

HTTPError: 401 Client Error: Unauthorized for url: https://cws.correios.com.br/logisticaReversaWS/logisticaReversaService/logisticaReversaWS?wsdl

In [ ]:
request = RequestSolicitarPostagemReversa(
            codAdministrativo='13415611',
            codigo_servico='04677',
            cartao='0075199017',
            
            destinatario=Destinatario(
                nome='Nike - JP LOGISTICA',
                logradouro='Rodovia Fernão Dias',
                numero='S/N',
                complemento='Km 947.5 G40 Modulo B-NIVEL1',
                bairro='Bairro dos Pires',
                cidade='́Extrema',
                uf='MG',
                cep='37640950'
            ),
    
            coletas_solicitadas=[
                Coleta(
                    tipo='A',
                    valor_declarado=1500.00,
                    remetente=Remetente(
                        nome='Ciclano',
                        logradouro='Rua 35',
                        numero='10',
                        bairro='Águas Claras(Sul)',
                        cidade='Brasília',
                        uf='DF',
                        cep='71931-180',
                        ddd='61',
                        telefone='34262222',
                        email='ciclano@mail.com',
                        identificacao='',
                        ddd_celular='61',
                        celular='92236666',
                        sms='N'
                    ),

                    obj_col=[
                        Objeto(
                            item=1,
                            id='582298235'

                        )
                    ]
                )
            ]
        )

response = cliente.SolicitarPostagemReversa(request)
print(response)


In [ ]:
# ###############################################################################################################
# from unittest import TestCase
# from correios_lib.webservices import LogisticaReversa
# from correios_lib.entities import Remetente, Destinatario, Coleta, \
#      Objeto, Produto, ColetaSimultanea
# from correios_lib.requests import RequestSolicitarPostagemReversa, \
#      RequestAcompanharPedido, RequestAcompanharPedidoPorData, \
#      RequestCancelarPedido, RequestSolicitarRange, \
#      RequestCalcularDigitoVerificador, RequestSolicitarPostagemSimultanea
# import datetime


# # cliente = LogisticaReversa(
# #             env='HOMOLOG',
# #             id_correios='empresacws',
# #             password='123456',
# #             cert=False
# #         )


# request = RequestAcompanharPedidoPorData(
#     codAdministrativo='13415611',
#     tipoSolicitacao='A',
#     data=datetime.date(2022, 11, 14)
# )
# response = cliente.AcompanharPedidoPorData(request)
# print(response)